In [42]:
import numpy as np
import pandas as pd
from scipy.special import comb

champion_data = pd.read_csv('data/tftset6champions.csv')
rolling_probability_data = pd.read_json('data/rollprobability.json')
unit_pool_data = pd.read_json('data/unitpool.json', typ='series')

np.set_printoptions(linewidth=110)

In [45]:
UNIT_MAX = None
POOL_MAX = None
P_N_COST = None
DIST_N_COST = None

def config_rolldown(level, n_cost):
    global UNIT_MAX, POOL_MAX, P_N_COST, DIST_N_COST
    UNIT_MAX = unit_pool_data[n_cost]
    POOL_MAX = UNIT_MAX * champion_data[champion_data['cost'] == n_cost]['cost'].size
    P_N_COST = rolling_probability_data[level][n_cost - 1]
    DIST_N_COST = binary_dist(5, P_N_COST)
    
def binary_dist(n, p):
    prob_density = np.zeros(n + 1)
    for k in range(n + 1):
        prob_density[k] = comb(n, k) * np.power(p, k) * np.power(1 - p, n - k)
    return prob_density

# row is number of unit pulled from bag, column is number of hits
def bag_dist(n, pool, unit):
    prob_density = np.zeros((n + 1, n + 1))
    for i in range(prob_density.shape[0]):
        for j in range(prob_density.shape[1]):
            prob_density[i][j] = comb(unit, j) * comb(pool - unit, i - j) / comb(pool, i)
    return prob_density

def roll_node_naive(level, current, n_cost):
    bag_dist_array = bag_dist(5, POOL_MAX - current, UNIT_MAX - current)
    hit_dist_array = np.matmul(DIST_N_COST, bag_dist_array)
    return hit_dist_array

level = 8
n_cost = 4
config_rolldown(level, n_cost)
for current in np.arange(9):
    roll_node_dist = roll_node_naive(level, current, n_cost)
    print(current, roll_node_dist, np.sum(roll_node_dist))


0 [8.91043677e-01 1.04366440e-01 4.50051867e-03 8.85742814e-05 7.87641818e-07 2.50045021e-09] 0.9999999999999999
1 [8.99005297e-01 9.70968902e-02 3.82904094e-03 6.82274421e-05 5.42518942e-07 1.51602100e-09] 1.0
2 [9.07148309e-01 8.96012265e-02 3.19913356e-03 5.09736793e-05 3.56789963e-07 8.59734851e-10] 1.0
3 [9.15478658e-01 8.18704887e-02 2.61396776e-03 3.66635647e-05 2.21295751e-07 4.47062122e-10] 0.9999999999999999
4 [9.24002548e-01 7.38952807e-02 2.07692338e-03 2.51209975e-05 1.27124278e-07 2.06706143e-10] 0.9999999999999999
5 [9.32726444e-01 6.56657463e-02 1.59160342e-03 1.61404223e-05 6.57426366e-08 8.06658117e-11] 1.0000000000000002
6 [9.41657095e-01 5.71715420e-02 1.16184973e-03 9.48377369e-06 2.91502063e-08 2.39919393e-11] 0.9999999999999998
7 [9.50801544e-01 4.84018087e-02 7.91759867e-04 4.87712306e-06 1.00557951e-08 4.16389030e-12] 1.0
8 [9.60167145e-01 3.93451407e-02 4.85705311e-04 2.00699512e-06 2.08194515e-09 0.00000000e+00] 1.0


In [51]:
def roll_node_improved(level, current, current_contested, current_uncontested, n_cost):
    bag_dist_array = bag_dist(5, POOL_MAX - current - current_contested - current_uncontested, UNIT_MAX - current - current_contested)
    hit_dist_array = np.matmul(DIST_N_COST, bag_dist_array)
    return hit_dist_array

level = 8
n_cost = 4
config_rolldown(level, n_cost)
print("UNCONTESTED")
for current in np.arange(9):
    roll_node_dist = roll_node_improved(level, current, 0, 27, n_cost)
    print(current, roll_node_dist, np.sum(roll_node_dist))
print("CONTESTED")
for current in np.arange(9):
    roll_node_dist = roll_node_improved(level, current, 3, 27, n_cost)
    print(current, roll_node_dist, np.sum(roll_node_dist))

UNCONTESTED
0 [8.64516444e-01 1.28289320e-01 7.01688254e-03 1.75362590e-04 1.98244098e-06 8.00986253e-09] 1.0
1 [8.74066320e-01 1.19794607e-01 6.00167418e-03 1.36016477e-04 1.37719574e-06 4.90604080e-09] 1.0
2 [8.83889764e-01 1.10965424e-01 5.04155704e-03 1.02338814e-04 9.13631840e-07 2.81117489e-09] 0.9999999999999999
3 [8.93998102e-01 1.01784965e-01 4.14221930e-03 7.41398326e-05 5.71715522e-07 1.47730109e-09] 0.9999999999999998
4 [9.04403274e-01 9.22353877e-02 3.30983301e-03 5.11727351e-05 3.31402802e-07 6.90422503e-10] 1.0
5 [9.15117869e-01 8.22977348e-02 2.55109738e-03 3.31257435e-05 1.72969716e-07 2.72393253e-10] 1.0
6 [9.26155170e-01 7.19518533e-02 1.87328650e-03 1.96131335e-05 7.74170299e-08 8.19227829e-11] 1.0
7 [9.37529203e-01 6.11763035e-02 1.28430169e-03 1.01651227e-05 2.69626181e-08 1.43800630e-11] 1.0
8 [9.49254787e-01 4.99482615e-02 7.92729156e-04 4.21646454e-06 5.63698468e-09 0.00000000e+00] 1.0
CONTESTED
0 [8.93998102e-01 1.01784965e-01 4.14221930e-03 7.41398326e-05 5.7